In [ ]:
# ssh -f django@bcpp3.bhp.org.bw -L5003:localhost:3306 -N
# ssh -f django@bhp066.bhp.org.bw -L5002:localhost:3306 -N

In [21]:
# List of missing subject identifier
missing_arvs = []
file_name = '/Users/ckgathi/Downloads/BHP_FILE/check19.csv'
f = open(file_name, 'r')
lines = f.readlines()
lines.pop(0)
for line in lines:
    line = line.strip()
    missing_arvs.append(line)

missing_arvs = list(set(missing_arvs))
print(len(missing_arvs))

164


In [22]:
# List of id for missing avrs data for HivCareAdherence
from bcpp_subject.models import HivCareAdherence
from django_pandas.io import read_frame

hiv_care_adherence = HivCareAdherence.objects.using('bhp066').filter(subject_visit__subject_identifier__in=missing_arvs)
print(hiv_care_adherence.count())
df = read_frame(hiv_care_adherence)
hiv_care_adherence_ids = df['id'].tolist()
ids = []
for id in hiv_care_adherence_ids:
    ids.append(id.hex)
ids = list(set(ids))

313


In [26]:
import MySQLdb
import pandas as pd
db_name = 'bcpp015'
connection = MySQLdb.connect(host='127.0.0.1', port=5003,user='root', passwd='cc3721b', db=f'{db_name}')
data = []

count_found = 0
for idd in ids:
    sql = f"SELECT CONCAT(bcpp_subject_arv.short_name, bcpp_subject_hivcareadherence_arvs.hivcareadherence_id) as avr_id FROM bcpp_subject_arv INNER JOIN bcpp_subject_hivcareadherence_arvs ON bcpp_subject_arv.id = bcpp_subject_hivcareadherence_arvs.arv_id WHERE bcpp_subject_hivcareadherence_arvs.hivcareadherence_id = \'{idd}\';"
    df_arvs = pd.read_sql(sql, con=connection)
    if not df_arvs.empty:
        arvs_ids_list = df_arvs['avr_id'].tolist()
        if len(arvs_ids_list) > 1:
            count_found += len(arvs_ids_list)
        else:
            count_found += 1
        data.append(arvs_ids_list)
print(f"Found items: {count_found}")
print(data)

Found items: 11
[['Lopinavir/ritonavirac7cc51b37d14fe1a66174e3f2779fe0'], ['Lopinavir/ritonavir158bfd8990b54f0b9289c072a74ab53e'], ['Lopinavir/ritonavir7fa639271ec945f0b6985b2d65436ed0'], ['Lopinavir/ritonavir513401b4c9bd4b00853c693fa43021c6'], ['Lopinavir/ritonavirf657ef76c15545d6a9809f111b46d01c'], ['Lopinavir/ritonavirbad2f0b476124ceb91ae25d055643473'], ['Lopinavir/ritonavire345543f343e4134837815dfdfa63eb6'], ['Lopinavir/ritonavirb378bbd68cc143fcac81c2d332f92ecb'], ['Lopinavir/ritonavir12be8cb2914a4d4abdc1b242a48c2745'], ['Lopinavir/ritonavir006e8dcaea8e472f95285025421df923'], ['Lopinavir/ritonavir4e4646ba71b24cf49aafc702de72a781']]


In [27]:
# Update central server
from bcpp_subject.models import Arv
count_moved = 0
ccc = []
items = 0
for d in data:
    for hca_id in d:
        short_name = hca_id[:-32]
        obj_uuid = hca_id[-32:]
        try:
            hca = HivCareAdherence.objects.using('bhp066').get(id=obj_uuid)
        except HivCareAdherence.DoesNotExist:
            print(f'this is a problem it needs to be moved to the server {hca_id}')
        else:
            arv = Arv.objects.get(short_name=short_name)
            if not arv in hca.arvs.all():
                hca.arvs.add(arv)
                hca.save_base(using='bhp066', raw=True)
                count_moved += 1
                ccc.append(obj_uuid)
        items += 1

In [31]:
# Check if the data has been moved
mis_arv = 0
available = 0
for d in data:
    for hca_id in d:
        short_name = hca_id[:-32]
        obj_uuid = hca_id[-32:]
        try:
            hca = HivCareAdherence.objects.using('bhp066').get(id=obj_uuid)
        except HivCareAdherence.DoesNotExist:
            pass
        else:
            arv = Arv.objects.get(short_name=short_name)
            if not arv in hca.arvs.all():
                mis_arv += 1
            else:
                available += 1
print("Missing: ", mis_arv)            
print("Available: ", available)

Missing:  0
Available:  11
